# Liquidity Stress Tool

Interactive liquidity stress testing tool used to assess whether a fund
can meet redemptions under market stress.

The model operates on aggregated liquidity buckets, mirroring how liquidity
risk teams perform funding analysis.


In [12]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [13]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

from engine.assets import default_liquidity_profile
from engine.scenarios import build_scenario
from engine.liquidity import run_liquidity_stress
from engine.metrics import liquidity_metrics


In [14]:
redemption_slider = widgets.FloatSlider(
    value=0.10,
    min=0.0,
    max=0.5,
    step=0.01,
    description="Redemption %",
    readout_format=".0%"
)

freeze_t1 = widgets.Checkbox(
    value=False,
    description="Freeze T+1 Liquidity"
)

freeze_t5 = widgets.Checkbox(
    value=False,
    description="Freeze T+5 Liquidity"
)

controls = widgets.VBox([
    redemption_slider,
    freeze_t1,
    freeze_t5
])

display(controls)

In [15]:
output = widgets.Output()
display(output)

Output()

In [16]:
def run_stress(_=None):
    with output:
        clear_output()

        profile = default_liquidity_profile()

        freeze_buckets = []
        if freeze_t1.value:
            freeze_buckets.append("T+1 Liquidity")
        if freeze_t5.value:
            freeze_buckets.append("T+5 Liquidity")

        scenario = build_scenario(
            redemption_rate=redemption_slider.value,
            freeze_buckets=freeze_buckets
        )

        if scenario["freeze_buckets"]:
            profile.loc[
                profile["bucket"].isin(scenario["freeze_buckets"]),
                "available"
            ] = False

        waterfall, summary = run_liquidity_stress(
            profile=profile,
            redemption_rate=scenario["redemption_rate"]
        )

        metrics = liquidity_metrics(summary)

        print("Scenario")
        print(scenario)

        print("\nMetrics")
        for k, v in metrics.items():
            if isinstance(v, (int, float)):
                print(f"{k}: {round(v, 4)}")
            else:
                print(f"{k}: {v}")

        print("\nLiquidity Waterfall")
        display(waterfall)

In [17]:
redemption_slider.observe(run_stress, names="value")
freeze_t1.observe(run_stress, names="value")
freeze_t5.observe(run_stress, names="value")

run_stress()